In [ ]:
!pip install langchain-ollama langgraph
%pip install -qU duckduckgo-search langchain-community aiopyston
from pyston import PystonClient,File
import asyncio
from langchain_community.tools import DuckDuckGoSearchRun
from langgraph.prebuilt import create_react_agent
from langchain_ollama.chat_models import ChatOllama
from langgraph.graph import MessagesState

from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    ToolMessage,
    trim_messages,
)
from langchain_core.messages.utils import count_tokens_approximately


In [ ]:
recurse = 100 # model recursion limit, update if needed.
llm = ChatOllama(model="qwen2.5:1.5b")
search = DuckDuckGoSearchRun()
client = PystonClient()
privateState = {}

async def runCode(code:str):
    """
    sandboxed code environment
    args:
        - code (str): python code to be run
    returns:
        - result (str): the result of the code being run
    """    
    try:
        result = await client.execute("python", [File(code)])
        return result
    except Exception as e:
        return "There was an error: "+e

agent_prompt = """
You are a helpful synthetic agent.
"""

agent = create_react_agent(
    model=llm,
    tools=[search, runCode],
    prompt=agent_prompt
)

messages = []
user_input = input("User: ")
messages.append(HumanMessage(content=user_input, additional_kwargs={}, response_metadata={}))
result = await agent.ainvoke({"messages": messages})        
for msg in result["messages"]:
    msg.pretty_print()
